In [1]:
import numpy as np

from keras.models import load_model

from keras.models import Model#, Sequential
from keras.layers import Input, Dense, Conv1D, concatenate, Dropout
from keras.layers import GlobalMaxPooling1D,GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint, Callback, LearningRateScheduler
from keras.optimizers import Adam, Nadam,SGD

Using TensorFlow backend.


In [3]:
# LOAD MODEL

# model = load_model('/content/final_ViraMiner_afterFT.hdf5')
model = load_model('/content/best_ViraMiner_end2end.hdf5')
model.summary()

W0112 22:43:24.536756 139938234902400 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0112 22:43:24.554493 139938234902400 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0112 22:43:24.600919 139938234902400 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0112 22:43:24.601942 139938234902400 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0112 22:43:24.609081 139938234902400 dep

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 5)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 293, 1000)    41000       input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 290, 1200)    67200       input_1[0][0]                    
__________________________________________________________________________________________________
global_average_pooling1d_1 (Glo (None, 1000)         0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
global_max

In [0]:
# HELPER FUNCTIONS

def DNA_to_onehot(dna_line):
  options_onehot = {'A': [1,0,0,0,0],'C' :[0,1,0,0,0], 'G':[0,0,1,0,0] ,'T':[0,0,0,1,0],'N':[0,0,0,0,0]}
  onehot_data = map(lambda e: options_onehot[e], dna_line)
  onehot_data = np.array(onehot_data)
  return onehot_data 

def predict_one_by_one(model, seqs):
  prediction_data=np.empty([len(seqs),3]) # true_class, pred_class, proba, seq_length 
  predictions = np.empty([len(seqs)])
  scores = np.empty([len(seqs)])
  for i,seq in enumerate(seqs):
    p = model.predict(np.expand_dims(seq,axis=0))
    prediction = int(p>0.5)
    predictions[i] = prediction
    scores[i] = p
    prediction_data[i,0] = prediction
    prediction_data[i,1] = p
    prediction_data[i,2] = len(seq)
  # return prediction_data
  return predictions, scores

In [5]:
# TEST THE THING

seqs = []

# virus = 0
seq =  'AAAAAAACCAGCAAAAGACTAAACAATGTCTACATCATAATGTTGTTCTCTATTACCACCAAACATAGAGAAATCTAATGGTGCAGCACTATCTCCTCTTAAATCAAAGTCAATAACTTCTTCATGTCCATCCTCACCAACAGCAATACAAGTAGCTAATAAGAAGTCTGGACTTTCAGCATCCTCTTCACTTAATCCATTACAATCAACAACAGTATCTTCAGCAATATCATTCATAAAAGAATGTTTATCAAAGCATATTTCACCTCTAGCAACAGCATAGCATCTTTGAGCAAACTC'
seqs.append(DNA_to_onehot(seq))
# normal = 1
seq2 = 'AACAGTGGGCCACCCATATTTTGAAATACCGAATCCTGCTGAGGCTGGGAAAATAGCAGTTCCAAAGGTTTCTGGCTCTCAATATAGAGTATTCAGATGCAAATTACCAGATCCAAATAAATTTGCGTTAATAGATAGAACTGTTTATAATTCTGAAAGAGAGAGGTTAGTTTGGAAACTCAGAGGCTTACAATTAGGTCGTGGAGGTCCTTTAGGTGTTGGAACTAGTGGACATCCGTTATTTAATAAGGTTCTCGACACAGAAAATCCATCTACATATCCAGTTAAACAAACTGATGA'
seqs.append(DNA_to_onehot(seq2))

predictions, scores = predict_one_by_one(model, seqs)
scores

array([0.07366241, 0.58207309])

In [0]:
# RUN REAL DATA

seqs = []
lines = []

# normal unmapped
# f = open('/content/ZR421326-N.unmapped.fasta')
# tumor unmapped
# f = open('/content/ZR421326-T.unmapped.fasta')
# tumor mapped
f = open('/content/ZR421326-T.mapped.short.fasta')


# for line in f:
N = 100000
for i in range(N):
  line = f.next().strip()
  line = line.strip()
  if line.startswith('>') is True: continue
  if len(line) is not 100: continue
  seq = DNA_to_onehot(('N' * 100) + line + ('N' * 100))
  # seq = DNA_to_onehot(line * 3)
  seqs.append(seq)
  lines.append(line)
  # break

f.close()

predictions, scores = predict_one_by_one(model, seqs)

In [9]:
# OUTPUT THE UNIQUES

unique, counts = np.unique(predictions, return_counts=True)
result = dict(zip(unique, counts))
result

{0.0: 43072, 1.0: 293}

In [10]:
# Percentage

float(result.get(1))/(result.get(0)+result.get(1)) * 100

0.675660094546293

In [11]:
# grab top 10 and blast

virus_data = []
for i in range(len(predictions)):
  if predictions[i] > 0.5:
    virus_data.append((lines[i], scores[i]))
  # break

data = sorted(virus_data, key=lambda virus: virus[1])
data.reverse()

data[:100]


[('TGTAGACTGAGTAAAAATTATAAGCTTCAGTGGTTAACAGTGTAGCTTTCTAAACTGGTTTCTGTTACAGTATATTTACTTGAAGCAATTGGGGGGGTAC',
  0.8685897588729858),
 ('TGTAGACTGAGTAAAAATTATAAGCTTCAGTGGTTAACAGTGTAGCTTTCTAAACTGGTTTCTGTTACAGTATATTTACTTGAAGCAATTGGGGGGGTAC',
  0.8685897588729858),
 ('GTAGACTGAGTAAAAATTATAAGCTTCAGTGGTTAACAGTGTAGCTTTCTAAACTGGTTTCTGTTACAGTATATTTACTTGAAGCAATTGGGGGGGTACT',
  0.8499968647956848),
 ('GTAGACTGAGTAAAAATTATAAGCTTCAGTGGTTAACAGTGTAGCTTTCTAAACTGGTTTCTGTTACAGTATATTTACTTGAAGCAATTGGGGGGGTACT',
  0.8499968647956848),
 ('GTAGACTGAGTAAAAATTATAAGCTTCAGTGGTTAACAGTGTAGCTTTCTAAACTGGTTTCTGTTACAGTATATTTACTTGAAGCAATTGGGGGGGTACT',
  0.8499968647956848),
 ('TTGTAGACTGAGTAAAAATTATAAGCTTCAGTGGTTAACAGTGTAGCTTTCTAAACTGGTTTCTGTTACAGTATATTTACTTGAAGCAATTGGGGGGGTA',
  0.8487735986709595),
 ('TTGTAGACTGAGTAAAAATTATAAGCTTCAGTGGTTAACAGTGTAGCTTTCTAAACTGGTTTCTGTTACAGTATATTTACTTGAAGCAATTGGGGGGGTA',
  0.8487735986709595),
 ('GTTTGTAGACTGAGTAAAAATTATAAGCTTCAGTGGTTAACAGTGTAGCTTTCTAAACTGGTTTCTGTTACAGTATATTTACTTGAAGCAATTG

In [86]:
cowsay.cow('hello Onno .... ')
cowsay.cow('hi Cosmo .... ')

NameError: ignored

# TODO

Find out which viruses are in the reads
Score the amount of virus in it

